In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium.plugins import MarkerCluster
import math
from branca.element import Template

In [2]:
cust = pd.read_csv("hackathon/customers.csv")
geo = pd.read_csv("hackathon/geolocation.csv")
good = pd.read_csv('hackathon/good_cust_with_ord_id.csv')
bad = pd.read_csv('hackathon/bad_cust_with_ord_id.csv')
tot=pd.read_csv('hackathon/collected_dataset.csv')
ord_ = pd.read_csv("hackathon/orders.csv")
ord_rev = pd.read_csv("hackathon/order_reviews.csv")

In [5]:
tot

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,1.0,credit_card,...,58.0,598.0,4.0,650.0,28.0,9.0,14.0,27277.0,volta redonda,SP
1,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,1.0,credit_card,...,58.0,598.0,4.0,650.0,28.0,9.0,14.0,27277.0,volta redonda,SP
2,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,1.0,credit_card,...,58.0,598.0,4.0,650.0,28.0,9.0,14.0,27277.0,volta redonda,SP
3,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,1.0,credit_card,...,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,3471.0,sao paulo,SP
4,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,1.0,credit_card,...,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,3471.0,sao paulo,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357418,fffe18544ffabc95dfada21779c9644f,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25 00:00:00,1.0,credit_card,...,56.0,1306.0,1.0,100.0,20.0,20.0,20.0,4733.0,sao paulo,SP
357419,fffe18544ffabc95dfada21779c9644f,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25 00:00:00,1.0,credit_card,...,56.0,1306.0,1.0,100.0,20.0,20.0,20.0,4733.0,sao paulo,SP
357420,fffe41c64501cc87c801fd61db3f6244,96d649da0cc4ff33bb408b199d4c7dcf,delivered,2018-06-09 17:00:18,2018-06-09 17:10:13,2018-06-11 14:11:00,2018-06-14 17:56:26,2018-06-28 00:00:00,1.0,credit_card,...,47.0,511.0,1.0,600.0,30.0,3.0,19.0,14940.0,ibitinga,SP
357421,fffe41c64501cc87c801fd61db3f6244,96d649da0cc4ff33bb408b199d4c7dcf,delivered,2018-06-09 17:00:18,2018-06-09 17:10:13,2018-06-11 14:11:00,2018-06-14 17:56:26,2018-06-28 00:00:00,1.0,credit_card,...,47.0,511.0,1.0,600.0,30.0,3.0,19.0,14940.0,ibitinga,SP


In [8]:
good_geo = pd.merge(
    good, cust, on='customer_unique_id', how='left'
)

bad_geo = pd.merge(
    bad, cust, on='customer_unique_id', how='left'
)


city_distribution = good_geo['customer_city'].value_counts().reset_index()
city_distribution.columns = ['city', 'good_customer_count']

state_distribution = good_geo['customer_state'].value_counts().reset_index()
state_distribution.columns = ['state', 'good_customer_count']

## Карта распределения пользователей по штатам

In [9]:
customers_per_state = cust['customer_state'].value_counts().reset_index()
customers_per_state.columns = ['state', 'customer_count']

geo_agg = geo.groupby('geolocation_state').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean'
}).reset_index()

map_data = pd.merge(
    customers_per_state,
    geo_agg,
    left_on='state',
    right_on='geolocation_state',
    how='inner'
)

def scale_radius(count, max_count):
    scaled = 5 + 50 * math.sqrt(count / max_count)
    return min(scaled, 30)  

max_count = map_data['customer_count'].max()

m = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

for idx, row in map_data.iterrows():
    radius = scale_radius(row['customer_count'], max_count)
    
    folium.CircleMarker(
        location=[row['geolocation_lat'], row['geolocation_lng']],
        radius=radius,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        weight=1
    ).add_to(m)
    
    folium.Marker(
        [row['geolocation_lat'], row['geolocation_lng']],
        icon=folium.DivIcon(
            html=f'''
            <div style="
                font-size: {max(8, min(14, radius/2))}pt;
                color: black;
                font-weight: bold;
                text-align: center;
                width: {radius*2}px;
                line-height: {radius*2}px;
            ">{row["customer_count"]}</div>
            ''',
            icon_size=(radius*2, radius*2),  
            icon_anchor=(radius, radius)      
        )
    ).add_to(m)

m

## Распределение "хороших"покупателей

In [10]:
good_per_state = good_geo['customer_state'].value_counts().reset_index()
good_per_state.columns = ['state', 'customer_count']

good_map_data = pd.merge(
    good_per_state,
    geo_agg,
    left_on='state',
    right_on='geolocation_state',
    how='inner'
)

def scale_radius_g(count, max_count):
    scaled = 5 + 100 * math.sqrt(count / max_count)
    return min(scaled, 30) 

m1 = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

for idx, row in good_map_data.iterrows():
    radius = scale_radius_g(row['customer_count'], max_count)
    
    folium.CircleMarker(
        location=[row['geolocation_lat'], row['geolocation_lng']],
        radius=radius,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        weight=1
    ).add_to(m1)
    
    folium.Marker(
        [row['geolocation_lat'], row['geolocation_lng']],
        icon=folium.DivIcon(
            html=f'''
            <div style="
                font-size: {max(8, min(14, radius/2))}pt;
                color: black;
                font-weight: bold;
                text-align: center;
                width: {radius*2}px;
                line-height: {radius*2}px;
            ">{row["customer_count"]}</div>
            ''',
            icon_size=(radius*2, radius*2),  
            icon_anchor=(radius, radius)      
        )
    ).add_to(m1)

m1

## Распределение "плохих" покупателей

In [11]:
bad_per_state = bad_geo['customer_state'].value_counts().reset_index()
bad_per_state.columns = ['state', 'customer_count']

bad_map_data = pd.merge(
    bad_per_state,
    geo_agg,
    left_on='state',
    right_on='geolocation_state',
    how='inner'
)

m1 = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

for idx, row in bad_map_data.iterrows():
    radius = scale_radius_g(row['customer_count'], max_count)
    
    folium.CircleMarker(
        location=[row['geolocation_lat'], row['geolocation_lng']],
        radius=radius,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        weight=1
    ).add_to(m1)
    
    folium.Marker(
        [row['geolocation_lat'], row['geolocation_lng']],
        icon=folium.DivIcon(
            html=f'''
            <div style="
                font-size: {max(8, min(14, radius/2))}pt;
                color: black;
                font-weight: bold;
                text-align: center;
                width: {radius*2}px;
                line-height: {radius*2}px;
            ">{row["customer_count"]}</div>
            ''',
            icon_size=(radius*2, radius*2),  
            icon_anchor=(radius, radius)      
        )
    ).add_to(m1)

m1

In [ ]:
sum_good = good_per_state['customer_count'].sum()
sum_bad = bad_per_state['customer_count'].sum()
print("Всего хороших:",sum_good, '\nВсего плохих:', sum_bad)

## Соотношение покупателей

In [13]:

# Объединяем данные
merged = pd.merge(
    good_per_state.rename(columns={'customer_count': 'good'}),
    bad_per_state.rename(columns={'customer_count': 'bad'}),
    on='state',
    how='outer'
).fillna(0)

# Добавляем суммарное количество и доли
merged['total'] = merged['good'] + merged['bad']
merged['good_ratio'] = merged['good'] / merged['total']
merged['bad_ratio'] = merged['bad'] / merged['total']

geo_data = geo.groupby('geolocation_state').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean'
}).reset_index()

final_data = pd.merge(
    merged,
    geo_data,
    left_on='state',
    right_on='geolocation_state',
    how='inner'
)

m = folium.Map(
    location=[-14.2, -51.9],
    zoom_start=4,
    max_bounds=True,
    min_lon=-73.9,
    max_lon=-34.8,
    min_lat=-33.8,
    max_lat=5.3
)

def add_pie_marker(row, map_obj):
    try:
        total = row['total']
        if total == 0:
            return
            
        max_total = final_data['total'].max()
        radius = 10 + 30 * math.sqrt(total / max_total)
        good_angle = 360 * row['good_ratio']
        
        svg = f"""
        <svg width="{radius*2}" height="{radius*2}" viewBox="0 0 {radius*2} {radius*2}">
            <path d="M{radius},{radius} L{radius*2},{radius} 
                     A{radius},{radius} 0 {1 if good_angle > 180 else 0},1 
                     {radius + radius*math.cos(math.radians(good_angle))},
                     {radius + radius*math.sin(math.radians(good_angle))} Z"
                  fill="#4CAF50" stroke="none"/>
            <path d="M{radius},{radius} 
                     L{radius + radius*math.cos(math.radians(good_angle))},
                     {radius + radius*math.sin(math.radians(good_angle))}
                     A{radius},{radius} 0 {1 if (360-good_angle) > 180 else 0},1 
                     {radius*2},{radius} Z"
                  fill="#F44336" stroke="none"/>
            <circle cx="{radius}" cy="{radius}" r="{radius*0.6}" fill="white" />
            <text x="{radius}" y="{radius}" 
                  font-size="{max(8, min(12, radius//2))}" 
                  text-anchor="middle" dominant-baseline="middle"
                  font-weight="bold" fill="#333">{int(total)}</text>
        </svg>
        """
        
        popup_content = f"""
        <b>{row['state']}</b><br>
        Всего: {int(total)}<br>
        Хорошие: {int(row['good'])} ({row['good_ratio']:.1%})<br>
        Плохие: {int(row['bad'])} ({row['bad_ratio']:.1%})
        """
        popup = folium.Popup(popup_content, max_width=250)
        
        marker = folium.Marker(
            location=[row['geolocation_lat'], row['geolocation_lng']],
            icon=folium.DivIcon(
                html=svg,
                icon_size=(radius*2, radius*2),
                icon_anchor=(radius, radius)
            ),
            popup=popup 
        )
        marker.add_to(map_obj)
        
    except Exception as e:
        print(f"Ошибка при обработке штата {row['state']}: {str(e)}")

#Добавляем круги на карту
for _, row in final_data.iterrows():
    add_pie_marker(row, m)

#Добавляем легенду
legend_html = """
<div style="
    position: fixed; 
    bottom: 50px;
    left: 50px;
    width: 180px;
    padding: 10px;
    background: white;
    border-radius: 5px;
    box-shadow: 0 0 5px rgba(0,0,0,0.2);
    z-index: 9999;
">
    <div style="display: flex; align-items: center;">
        <div style="width: 15px; height: 15px; background: green; margin-right: 5px;"></div>
        <span>Хорошие покупатели</span>
    </div>
    <div style="display: flex; align-items: center; margin-top: 5px;">
        <div style="width: 15px; height: 15px; background: red; margin-right: 5px;"></div>
        <span>Плохие покупатели</span>
    </div>
    <div style="margin-top: 10px; font-size: 12px;">
        Размер круга ≈ общему количеству
    </div>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Сохраняем и отображаем
m.save('customer_ratio_map.html')
m

In [ ]:
m.save('brazil_customers_ratio_map.html')
display(m) 

In [15]:
import sys
!{sys.executable} -m pip install selenium

  Using cached selenium-4.31.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached exceptiongroup-1.2.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 9.4/9.4 MB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 499.2/499.2 kB 2.4 MB/s eta 0:00:00
   ---------------------------------


[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import io
from PIL import Image
import selenium


In [17]:

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('map_good_bad.png')